Tracking social empowerment schemes 

output should be visualization of reach, progress, and updates of various government welfare schemes

In [1]:
# Import Libraries
import dash
from dash import dcc, html
from dash import dcc, html, dash_table as DataTable
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px

# Step 1: Load the dataset
df = pd.read_csv('Centerdata.csv')

# Step 2: Clean the data
numeric_columns = [
    "Revenue", "Actuals2019-2020 - Capital", "Actuals2019-2020 - Total",
    "Budget Estimates2020-2021 - Revenue", "Budget Estimates2020-2021 - Capital", "Budget Estimates2020-2021 - Total",
    "Revised Estimates2020-2021 - Revenue", "Revised Estimates2020-2021 - Capital", "Revised Estimates2020-2021 - Total",
    "Budget Estimates2021-2022 - Revenue", "Budget Estimates2021-2022 - Capital", "Budget Estimates2021-2022 - Total"
]

# Convert numeric columns and handle errors
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Replace NA with 0 and clip negative values
df.fillna(0, inplace=True)
df[numeric_columns] = df[numeric_columns].clip(lower=0)

# Step 3: Create a Dash app with a Bootstrap theme
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

# Helper function to create cards for key metrics
def create_card(title, value, color):
    return dbc.Card(
        dbc.CardBody([
            html.H5(title, className="card-title"),
            html.H2(f"{value:,.2f}", className="card-text", style={"color": color})
        ]),
        className="mb-4"
    )

# Step 4: Layout of the Dash App
app.layout = dbc.Container([
    # Title Section
    dbc.Row([
        dbc.Col(html.H1("Tracking Social Empowerment Schemes", className="text-center text-primary mb-4"), width=12)
    ]),

    # Overview Section with Metrics
    dbc.Row([
        dbc.Col(create_card("Total Revenue (2019-2020)", df["Revenue"].sum(), "green"), width=4),
        dbc.Col(create_card("Total Budget (2020-2021)", df["Budget Estimates2020-2021 - Total"].sum(), "blue"), width=4),
        dbc.Col(create_card("Total Budget (2021-2022)", df["Budget Estimates2021-2022 - Total"].sum(), "orange"), width=4),
    ]),

    # Dropdown for Scheme Selection
    dbc.Row([
        dbc.Col([
            html.Label("Select a Scheme:"),
            dcc.Dropdown(
                id="scheme-dropdown",
                options=[{"label": scheme, "value": scheme} for scheme in df["Scheme"].unique()],
                value=df["Scheme"].iloc[0],
                placeholder="Select a Scheme",
                clearable=True
            )
        ], width=6),
    ], className="mb-4"),

    # Graph Section
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="revenue-bar-chart",
                config={"displayModeBar": True}
            )
        ], width=120),
        
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="budget-trend-line-chart",
                config={"displayModeBar": True}
            )
        ], width=12),
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="scatter-revenue-vs-capital",
                config={"displayModeBar": True}
            )
        ], width=6),
        dbc.Col([
            dcc.Graph(
                id="stacked-bar-actuals-vs-revised",
                config={"displayModeBar": True}
            )
        ], width=6),
    ]),

    # Data Table Section
    dbc.Row([
        dbc.Col([
            html.H4("Interactive Data Table", className="mt-4"),
            DataTable.DataTable(
                id="data-table",
                columns=[
                    {"name": col, "id": col, "type": "numeric" if df[col].dtype in ['float64', 'int64'] else "text"} 
                    for col in df.columns
                ],
                data=df.to_dict("records"),
                page_size=10,
                filter_action="native",
                sort_action="native",
                style_table={'overflowX': 'auto', 'margin-top': '20px'},
                style_cell={
                    'textAlign': 'left',
                    'padding': '5px',
                    'backgroundColor': '#f8f9fa',
                    'color': '#212529',
                    'fontFamily': 'Arial, sans-serif',
                },
                style_header={
                    'backgroundColor': '#343a40',
                    'color': 'white',
                    'fontWeight': 'bold',
                    'textAlign': 'center',
                },
                style_data_conditional=[
                    {
                        'if': {
                            'filter_query': '{{{col}}} > 100000'.format(col="Revenue"),
                            'column_id': 'Revenue'
                        },
                        'backgroundColor': '#ffcccb',
                        'color': '#8b0000',
                        'fontWeight': 'bold',
                    },
                    {
                        'if': {'row_index': 'odd'},
                        'backgroundColor': '#f2f2f2',
                    },
                ],
                tooltip_data=[
                    {
                        col: {'value': str(value), 'type': 'markdown'}
                        for col, value in row.items()
                    } for row in df.to_dict('records')
                ],
                tooltip_duration=None,
            )
        ], width=12)
    ])
], fluid=True)

# Step 5: Callbacks for Graphs
@app.callback(
    dash.dependencies.Output("revenue-bar-chart", "figure"),
    [dash.dependencies.Input("scheme-dropdown", "value")]
)
def update_bar_chart(selected_scheme):
    filtered_df = df if not selected_scheme else df[df["Scheme"] == selected_scheme]
    fig = px.bar(
        filtered_df,
        x="Scheme",
        y=["Revenue", "Budget Estimates2020-2021 - Revenue", "Budget Estimates2021-2022 - Revenue"],
        title="Revenue vs Budget Estimates",
        labels={"value": "Amount (in Crores)", "Scheme": "Scheme"},
        barmode="group",
        color_discrete_sequence=["#FF6347", "#4682B4", "#FFD700"]
    )
    fig.update_layout(transition_duration=500)
    return fig


# @app.callback(
#     dash.dependencies.Output("budget-pie-chart", "figure"),
#     [dash.dependencies.Input("scheme-dropdown", "value")]
# )
# def update_pie_chart(selected_scheme):
#     filtered_df = df if not selected_scheme else df[df["Scheme"] == selected_scheme]
#     fig = px.pie(
#         filtered_df,
#         values="Budget Estimates2020-2021 - Total",
#         names="Scheme",
#         title="Budget Distribution by Scheme (2020-2021)",
#         color_discrete_sequence=px.colors.sequential.RdBu
#     )
#     return fig


@app.callback(
    dash.dependencies.Output("budget-trend-line-chart", "figure"),
    [dash.dependencies.Input("scheme-dropdown", "value")]
)
def update_line_chart(selected_scheme):
    filtered_df = df if not selected_scheme else df[df["Scheme"] == selected_scheme]
    fig = px.line(
        filtered_df,
        x="Scheme",
        y=["Actuals2019-2020 - Total", "Budget Estimates2020-2021 - Total", "Budget Estimates2021-2022 - Total"],
        title="Budget Trends Over Years",
        labels={"value": "Amount (in Crores)", "Scheme": "Scheme"},
        line_shape="linear",
        color_discrete_sequence=["#6A5ACD", "#FF1493", "#1E90FF"]
    )
    fig.update_layout(transition_duration=500)
    return fig


@app.callback(
    dash.dependencies.Output("scatter-revenue-vs-capital", "figure"),
    [dash.dependencies.Input("scheme-dropdown", "value")]
)
def update_scatter_chart(selected_scheme):
    filtered_df = df if not selected_scheme else df[df["Scheme"] == selected_scheme]
    fig = px.scatter(
        filtered_df,
        x="Revenue",
        y="Actuals2019-2020 - Capital",
        title="Revenue vs Capital Expenditure (2019-2020)",
        color="Scheme",
        size="Actuals2019-2020 - Total",
        hover_name="Scheme",
        color_discrete_sequence=px.colors.qualitative.Plotly
    )
    return fig


@app.callback(
    dash.dependencies.Output("stacked-bar-actuals-vs-revised", "figure"),
    [dash.dependencies.Input("scheme-dropdown", "value")]
)
def update_stacked_bar_chart(selected_scheme):
    filtered_df = df if not selected_scheme else df[df["Scheme"] == selected_scheme]
    fig = px.bar(
        filtered_df,
        x="Scheme",
        y=["Actuals2019-2020 - Total", "Revised Estimates2020-2021 - Total", "Budget Estimates2021-2022 - Total"],
        title="Actuals vs Revised Estimates vs Budgets",
        labels={"value": "Amount (in Crores)", "Scheme": "Scheme"},
        barmode="stack",
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    return fig

# Step 6: Run the app
if __name__ == "__main__":
    app.run_server(debug=True,port=8051)
